In [1]:
from spektral.data import Dataset, Graph, BatchLoader
import numpy as np
from graphIO import read_adj_matrices_from_directory
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input
from spektral.layers import GlobalSumPool, ECCConv, AsymCheegerCutPool
import tensorflow as tf

In [2]:
DATA_DIR = 'C:/Users/mosta/OneDrive - UNCG/Academics/CSC 699 - Thesis/data/data/'
CURV_DIR = f'{DATA_DIR}/curv'
ADJ_DIR = f'{DATA_DIR}/adj'
AD_LABEL = [1]
CN_LABEL = [0]

In [3]:
ad_adj_matrices = read_adj_matrices_from_directory(f'{ADJ_DIR}/ad')
cn_adj_matrices = read_adj_matrices_from_directory(f'{ADJ_DIR}/cn')
print(f'Loaded {len(ad_adj_matrices)} AD and {len(cn_adj_matrices)} CN matrices')

Reading adjacency matrices: 100%|██████████| 50/50 [00:00<00:00, 384.93it/s]

Loaded 50 AD and 50 CN matrices


In [4]:
ad_curv_matrices = read_adj_matrices_from_directory(f'{CURV_DIR}/ad')
cn_curv_matrices = read_adj_matrices_from_directory(f'{CURV_DIR}/cn')
print(f'Loaded {len(ad_curv_matrices)} AD and {len(cn_curv_matrices)} CN matrices')

Reading adjacency matrices: 100%|██████████| 50/50 [00:00<00:00, 330.46it/s]

Loaded 50 AD and 50 CN matrices


In [5]:
class ConnectimeGraphDataset(Dataset):
    def __init__(self, adj_matrices, edge_features, labels, **kwargs):
        self.adj_matrices = adj_matrices
        self.edge_features = edge_features
        self.labels = labels
        super().__init__(**kwargs)
    def read(self):
        data_list = []
        for key in self.adj_matrices.keys():
            adj = self.adj_matrices[key]
            edge_feat = self.edge_features[key].reshape(self.edge_features[key].shape[0], self.edge_features[key].shape[1], 1)
            label = self.labels[key]
            data_list.append(Graph(x=np.eye(adj.shape[0]), a=adj, e=edge_feat, y=label))
        return data_list

In [6]:
adj_matrices = {**ad_adj_matrices, **cn_adj_matrices}
curv_matrices = {**ad_curv_matrices, **cn_curv_matrices}
labels = {**{key: AD_LABEL for key in ad_adj_matrices.keys()}, **{key: CN_LABEL for key in cn_adj_matrices.keys()}}

dataset = ConnectimeGraphDataset(adj_matrices, curv_matrices, labels)
dataset[0]

Graph(n_nodes=160, n_node_features=160, n_edge_features=1, n_labels=1)

In [7]:
class MyFirstGNN(Model):
    def __init__(self, n_hidden, n_labels):
        super().__init__()
        self.input_layer = ECCConv(n_hidden)
        self.pool = GlobalSumPool()
        # self.hidden_layer_1 = Dense(n_hidden, 'relu')
        # self.hidden_layer_2 = Dense(n_hidden, 'relu')
        self.output_layer = Dense(n_labels, activation='sigmoid')
    def call(self, inputs):
        out = self.input_layer(inputs)
        out = self.pool(out)
        # out = self.hidden_layer_1(out)
        # out = self.hidden_layer_2(out)
        out = self.output_layer(out)
        return out

In [8]:
loader = BatchLoader(dataset, batch_size=1, shuffle=True)

In [9]:
opt =  tf.keras.optimizers.Adam()
loss = tf.keras.losses.BinaryCrossentropy()

In [10]:
model = MyFirstGNN(128, dataset.n_labels)
model.compile(optimizer=opt,
              loss=loss,
              metrics=[tf.keras.metrics.BinaryAccuracy(), 
                       tf.keras.metrics.Precision(),
                       tf.keras.metrics.Recall(),
                       tf.keras.metrics.AUC()])

In [11]:
history = model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=10)

c:\Users\mosta\miniconda3\envs\brain_connectome\lib\site-packages\spektral\data\utils.py:221: UserWarning: you are shuffling a 'ConnectimeGraphDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


Epoch 1/10
100/100 [==============================] - 258s 3s/step - loss: 378142.3125 - binary_accuracy: 0.4300 - precision: 0.4255 - recall: 0.4000 - auc: 0.4300
Epoch 2/10
100/100 [==============================] - 258s 3s/step - loss: 260677.5625 - binary_accuracy: 0.5200 - precision: 0.5217 - recall: 0.4800 - auc: 0.5200
Epoch 3/10
100/100 [==============================] - 257s 3s/step - loss: 115569.2422 - binary_accuracy: 0.6200 - precision: 0.6200 - recall: 0.6200 - auc: 0.6200
Epoch 4/10
100/100 [==============================] - 257s 3s/step - loss: 21512.8105 - binary_accuracy: 0.7400 - precision: 0.7308 - recall: 0.7600 - auc: 0.7400
Epoch 5/10
100/100 [==============================] - 256s 3s/step - loss: 32007.0098 - binary_accuracy: 0.7100 - precision: 0.7143 - recall: 0.7000 - auc: 0.7100
Epoch 6/10
100/100 [==============================] - 257s 3s/step - loss: 16332.6387 - binary_accuracy: 0.6600 - precision: 0.6600 - recall: 0.6600 - auc: 0.6600
Epoch 7/10
100/100 

In [12]:
model.summary()

Model: "my_first_gnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ecc_conv (ECCConv)          multiple                  61568     
                                                                 
 global_sum_pool (GlobalSumP  multiple                 0         
 ool)                                                            
                                                                 
 dense (Dense)               multiple                  0 (unused)
                                                                 
 dense_1 (Dense)             multiple                  0 (unused)
                                                                 
 dense_2 (Dense)             multiple                  129       
                                                                 
Total params: 61,697
Trainable params: 61,697
Non-trainable params: 0
__________________________________________________

In [13]:
loss = model.evaluate(loader.load(), steps=loader.steps_per_epoch)

print('Test loss: {}'.format(loss))

100/100 [==============================] - 136s 1s/step - loss: 406.4082 - binary_accuracy: 0.9100 - precision: 0.9200 - recall: 0.9020 - auc: 0.9102
Test loss: [406.408203125, 0.9100000262260437, 0.9200000166893005, 0.9019607901573181, 0.9101641178131104]
